In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
import pandas as pd
import numpy as np
import keras
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

import sklearn.metrics as metrics
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)

In [ ]:
import os
import pandas as pd

healthy_dirs = [
    r'/kaggle/input/osteoporosis-database/Osteoporosis Knee X-ray/normal',
    r'/kaggle/input/osteoporosis-knee-xray-dataset/normal/normal',
    r'/kaggle/input/osteoporosis-knee-dataset-preprocessed128x256/Osteoporosis Knee Dataset/Normal',
    r'/kaggle/input/osteoporosis/osteoporosis/normal'
]


osteoporosis_dirs = [
    r'/kaggle/input/osteoporosis-database/Osteoporosis Knee X-ray/osteoporosis',
    r'/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis',
    r'/kaggle/input/osteoporosis-knee-dataset-preprocessed128x256/Osteoporosis Knee Dataset/Osteoporosis',
    r'/kaggle/input/osteoporosis/osteoporosis/osteoporosis'
]

filepaths = []
labels = []
dict_lists = [healthy_dirs, osteoporosis_dirs]
class_labels = ['Healthy', 'Osteoporosis']

for i, dir_list in enumerate(dict_lists):
    for j in dir_list:
        flist = os.listdir(j)
        for f in flist:
            fpath = os.path.join(j, f)
            filepaths.append(fpath)
            labels.append(class_labels[i])

Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
knee_osteoporosis_data = pd.concat([Fseries, Lseries], axis=1)
knee_osteoporosis_df = pd.DataFrame(knee_osteoporosis_data)
print(knee_osteoporosis_df.head())
print(knee_osteoporosis_df["labels"].value_counts())


In [ ]:
knee_osteoporosis_df.shape

In [ ]:
train_images, test_images = train_test_split(knee_osteoporosis_df, test_size=0.3, random_state=42)
train_set, val_set = train_test_split(knee_osteoporosis_df, test_size=0.2, random_state=42)

In [ ]:
print(train_set.shape)
print(test_images.shape)
print(val_set.shape)
print(train_images.shape)

In [ ]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=4,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=4,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=4,
                                    shuffle=False
                                   )

In [ ]:
classes=list(train.class_indices.keys())
print (classes)

In [ ]:
def show_knee_images(image_gen):
    test_dict = test.class_indices
    classes = list(test_dict.keys())
    images, labels=next(image_gen) # get a sample batch from the generator
    plt.figure(figsize=(20,20))
    length = len(labels)
    if length<25:
        r=length
    else:
        r=25
    for i in range(r):
        plt.subplot(5,5,i+1)
        image=(images[i]+1)/2 #scale images between 0 and 1
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color="green",fontsize=16)
        plt.axis('off')
    plt.show()

In [ ]:
show_knee_images(train)

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from keras.models import Model
from keras.applications.vgg19 import VGG19
import tensorflow as tf

vgg19_weights = '/kaggle/input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create the custom model by adding layers
x = vgg19_model.output

# Add your custom layers according to your specified architecture
x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(256, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = Dropout(0.5)(x) 
x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = Dropout(0.5)(x) 
x = Conv2D(512, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)

x = Flatten()(x)

x = Dense(4096, activation="relu")(x)
x = Dense(4096, activation="relu")(x)
x = Dense(4096, activation="relu")(x)

output = Dense(2, activation="sigmoid")(x)

# Create the custom model
model = Model(inputs=vgg19_model.input, outputs=output)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()


In [ ]:
history = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
history1 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history2 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history3 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history4 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history5 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history6 = model.fit(train, epochs=10, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history7 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history8 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9513-01813.h5")

In [ ]:
history8 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history9 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9406-01328.h5")

In [ ]:
history10 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9406-01093.h5")

In [ ]:
history11 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9510-01134.h5")

In [ ]:
history12 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history12 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history13 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9533-01016.h5")

In [ ]:
history14 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history15 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history16 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history17 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9533-00959.h5")

In [ ]:
history18 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9597-00927.h5")

In [ ]:
history19 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history20 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history21 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history22 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9618-0854.h5")

In [ ]:
history23 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history24 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history25 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history26 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
history27 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history28 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history29 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9682-00660.h5")

In [ ]:
history30 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history31 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
history32 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
history33 = model.fit(train, epochs=5, validation_data=val, verbose=1)

In [ ]:
model.evaluate(test, verbose=1)

In [ ]:
model.save("osteoporosisEEEEEEEEEEEEEEbinary-9745-00681.h5")

In [ ]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1) #pick class with highest  probability

labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred2 = [labels[k] for k in pred]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_test = test_images.labels # set y_test to the expected output
print(classification_report(y_test, pred2))
print("Accuracy of the Model:","{:.1f}%".format(accuracy_score(y_test, pred2)*100))

In [ ]:
plt.plot(history.history["accuracy"] +
history1.history["accuracy"] +
history2.history["accuracy"] +
history3.history["accuracy"] +
history4.history["accuracy"] +
history5.history["accuracy"] +
history6.history["accuracy"] +
history7.history["accuracy"] +
history8.history["accuracy"] +
history9.history["accuracy"] +
history10.history["accuracy"] +
history11.history["accuracy"] +
history12.history["accuracy"] +
history13.history["accuracy"] +
history14.history["accuracy"] +
history15.history["accuracy"] +
history16.history["accuracy"] +
history17.history["accuracy"] +
history18.history["accuracy"] +
history19.history["accuracy"] +
history20.history["accuracy"] +
history21.history["accuracy"] +
history22.history["accuracy"] +
history23.history["accuracy"] +
history24.history["accuracy"] +
history25.history["accuracy"] +
history26.history["accuracy"] +
history27.history["accuracy"] +
history28.history["accuracy"] +
history29.history["accuracy"] +
history30.history["accuracy"] +
history31.history["accuracy"] +
history32.history["accuracy"])



plt.plot(history.history["val_accuracy"] +
history1.history["val_accuracy"] +
history2.history["val_accuracy"] +
history3.history["val_accuracy"] +
history4.history["val_accuracy"] +
history5.history["val_accuracy"] +
history6.history["val_accuracy"] +
history7.history["val_accuracy"] +
history8.history["val_accuracy"] +
history9.history["val_accuracy"] +
history10.history["val_accuracy"] +
history11.history["val_accuracy"] +
history12.history["val_accuracy"] +
history13.history["val_accuracy"] +
history14.history["val_accuracy"] +
history15.history["val_accuracy"] +
history16.history["val_accuracy"] +
history17.history["val_accuracy"] +
history18.history["val_accuracy"] +
history19.history["val_accuracy"] +
history20.history["val_accuracy"] +
history21.history["val_accuracy"] +
history22.history["val_accuracy"] +
history23.history["val_accuracy"] +
history24.history["val_accuracy"] +
history25.history["val_accuracy"] +
history26.history["val_accuracy"] +
history27.history["val_accuracy"] +
history28.history["val_accuracy"] +
history29.history["val_accuracy"] +
history30.history["val_accuracy"] +
history31.history["val_accuracy"] +
history32.history["val_accuracy"])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history["loss"] +
history1.history["loss"] +
history2.history["loss"] +
history3.history["loss"] +
history4.history["loss"] +
history5.history["loss"] +
history6.history["loss"] +
history7.history["loss"] +
history8.history["loss"] +
history9.history["loss"] +
history10.history["loss"] +
history11.history["loss"] +
history12.history["loss"] +
history13.history["loss"] +
history14.history["loss"] +
history15.history["loss"] +
history16.history["loss"] +
history17.history["loss"] +
history18.history["loss"] +
history19.history["loss"] +
history20.history["loss"] +
history21.history["loss"] +
history22.history["loss"] +
history23.history["loss"] +
history24.history["loss"] +
history25.history["loss"] +
history26.history["loss"] +
history27.history["loss"] +
history28.history["loss"] +
history29.history["loss"] +
history30.history["loss"] +
history31.history["loss"] +
history32.history["loss"])


plt.plot(history.history["val_loss"] +
history1.history["val_loss"] +
history2.history["val_loss"] +
history3.history["val_loss"] +
history4.history["val_loss"] +
history5.history["val_loss"] +
history6.history["val_loss"] +
history7.history["val_loss"] +
history8.history["val_loss"] +
history9.history["val_loss"] +
history10.history["val_loss"] +
history11.history["val_loss"] +
history12.history["val_loss"] +
history13.history["val_loss"] +
history14.history["val_loss"] +
history15.history["val_loss"] +
history16.history["val_loss"] +
history17.history["val_loss"] +
history18.history["val_loss"] +
history19.history["val_loss"] +
history20.history["val_loss"] +
history21.history["val_loss"] +
history22.history["val_loss"] +
history23.history["val_loss"] +
history24.history["val_loss"] +
history25.history["val_loss"] +
history26.history["val_loss"] +
history27.history["val_loss"] +
history28.history["val_loss"] +
history29.history["val_loss"] +
history30.history["val_loss"] +
history31.history["val_loss"] +
history32.history["val_loss"])


plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Define the class labels
class_labels = ['Healthy','Osteoporosis']

# Calculate the confusion matrix
cm = confusion_matrix(y_test, pred2)

# Create a figure and plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 5))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues')

# Set tick labels and axis labels
plt.xticks(ticks=[0.5, 1.5], labels=class_labels)
plt.yticks(ticks=[0.5, 1.5], labels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("Actual")

# Set the title
plt.title("Confusion Matrix")

# Show the plot
plt.show()
